# M6 Project

In this project you will compare the performance of LDA and SVM for face recognition. You will use the Olivetti faces dataset, which contains 400 64x64 images from 40 different subjects, and your task is to discover the identity of a given face image. Some of these images are illustrated below.

<img src="m6project.png" width="400"/>

An initial version of the code with the problem specification (below) and a report template are available (at the bottom). Deliverables are the final code (non-functioning code is worth 0 points) and the comparison report.

Solve the task above using:
- LDA (20pts)
- SVM (20pts)

For LDA:
- Visualize a 2D representation of the faces in the dataset (20pts)

Split the dataset so that the first 5 images per subject are used for training, and the last 5 images are used for testing. If you need a validation set, use part of your training data. Compare the performance of LDA and SVM in terms of:
- Average F-Score (15pts)
- Confusion matrix (15pts)
- Visualize the individuals with highest confusion and check if they look alike (10pts)

# Implementation

You are free to change the code below as needed.

In [ ]:
from sklearn.datasets import fetch_olivetti_faces
import matplotlib.pyplot as plt
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn import svm
from sklearn.metrics import f1_score, confusion_matrix

In [ ]:
# Get the dataset

faces = fetch_olivetti_faces()

_, img_height, img_width = faces.images.shape

print(faces.images.shape)

In [ ]:
# Split the dataset

N_IDENTITIES = len(np.unique(faces.target)) # how many different individuals are in the dataset
GALLERY_SIZE = 5                            # use the first GALLERY_SIZE images per individual for training, the rest for testing

gallery_indices = []
probe_indices = []
for i in range(N_IDENTITIES):
    indices = list(np.where(faces.target == i)[0])
    gallery_indices += indices[:GALLERY_SIZE]
    probe_indices += indices[GALLERY_SIZE:]

x_train = faces.images[gallery_indices].reshape(-1, img_height*img_width) # vectorize train images
y_train = faces.target[gallery_indices]
x_test = faces.images[probe_indices].reshape(-1, img_height*img_width)    # vectorize test images
y_test = faces.target[probe_indices]

print(x_train.shape, x_test.shape)

In [ ]:
# training the LDA and SVM models then visualizing 2D LDA

lda = LDA(n_components=2)
x_train_lda = lda.fit_transform(x_train, y_train)
x_test_lda = lda.transform(x_test)

lda_clf = svm.SVC(kernel='linear')
lda_clf.fit(x_train_lda, y_train)

clf = svm.SVC(kernel='linear')
clf.fit(x_train, y_train)

plt.figure(figsize=(10,10))
for i in range(N_IDENTITIES):
    plt.scatter(x_train_lda[y_train==i, 0], x_train_lda[y_train==i, 1], label=str(i))
plt.legend(loc='best')
plt.title('LDA 2D representation')
plt.show()


# evaluating the performance of the models

y_pred_lda = lda_clf.predict(x_test_lda)
f_score_lda = f1_score(y_test, y_pred_lda, average='macro')
cm_lda = confusion_matrix(y_test, y_pred_lda)

y_pred_svm = clf.predict(x_test)
f_score_svm = f1_score(y_test, y_pred_svm, average='macro')
cm_svm = confusion_matrix(y_test, y_pred_svm)

print("LDA F-Score: ", f_score_lda)
print("SVM F-Score: ", f_score_svm)

print("LDA Confusion Matrix: \n", cm_lda)
print("SVM Confusion Matrix: \n", cm_svm)



# visualizing the individuals with the highest confusion

def plot_most_confused_individuals(cm, num):
    np.fill_diagonal(cm, 0)
    
    confused_ids = np.argsort(-cm.flatten())[:num]
    individuals = []
    for id in confused_ids:
        i, j = divmod(id, cm.shape[0])
        individuals.append((i, j))

    fig, axes = plt.subplots(1, num, figsize=(10, 10))
    for (i, j), ax in zip(individuals, axes):
        ax.imshow(faces.images[faces.target == j][0], cmap='gray')
        ax.set_title(f'Predicted: {j}, Actual: {i}', fontsize=10)
        ax.axis('off')
    plt.tight_layout() 
    plt.show()

plot_most_confused_individuals(cm_lda, 5)  # For LDA
plot_most_confused_individuals(cm_svm, 5)  # For SVM

In [ ]:
# Visualize image sets
def show_images(imgs, num_rows, num_cols):
    assert len(imgs) == num_rows*num_cols

    full = None
    for i in range(num_rows):
        row = None
        for j in range(num_cols):
            if row is None:
                row = imgs[i*num_cols+j].reshape(img_height, img_width)*255.0
            else:
                row = np.concatenate((row, imgs[i*num_cols+j].reshape(img_height, img_width)*255.0), axis=1)
        if full is None:
            full = row
        else:
            full = np.concatenate((full, row), axis=0)

    f = plt.figure(figsize=(num_cols, num_rows))
    plt.imshow(full, cmap='gray')
    plt.axis('off')
    plt.show()

print('TRAINING')
show_images(x_train, N_IDENTITIES, GALLERY_SIZE)
print('TESTING')
show_images(x_test, N_IDENTITIES, 10 - GALLERY_SIZE)

# Report template

## Visualization of the 2D face representation computed by LDA

I began the investigation by transforming the multi-dimensional facial recognition dataset into a 2D dataset using Linear Discriminant Analysis (LDA). This technique allowed me to reduce the dimensionality of the data while preserving as much of the original variance as possible. Consequently, I could visually represent the data and see how different individuals are grouped together.


## Experimental results

In the experiment, I utilized two different classifiers, the Linear Discriminant Analysis (LDA) and the Support Vector Machine (SVM) to predict identities based on the face recognition data. I then measured the performance of these two models using the F-score metric.

The F-score for the LDA classifier was 0.373, indicating a fair but not exceptional performance in recognizing faces. The LDA may have struggled due to the reduction of dimensions and the resulting loss of data complexity.

On the other hand, the SVM classifier showed an F-score of 0.9, significantly outperforming the LDA model. This suggests that the SVM was more effective in recognizing and differentiating individuals, even in the high-dimensional space.

The confusion matrices for both the LDA and SVM models further highlights the difference in their performance. The LDA matrix showed several misclassifications, indicating confusion between different individuals. In contrast, the SVM confusion matrix had more values along the diagonal and fewer misclassifications, reaffirming its superior performance in this instance.


## Analysis of the Results

While both LDA and SVM are powerful tools for face recognition, the experiment demonstrated that SVM had a distinct advantage over LDA in this particular task. The probable reason is that SVM works effectively with high dimensional data and finds the optimal plane that maximizes the margin between different classes. In contrast, LDA's performance may have been affected by the reduction in dimensions, causing it to lose some information crucial for accurately classifying faces.


## Visualization of the Individuals with Highest Confusion

Finally, I visualized the instances where the models had the most confusion, the individuals who were most frequently misclassified. These instances can provide useful insights into why these models may have made errors and help us refine our models in future iterations. We should look into these instances carefully to identify any common traits or anomalies that could have led to the misclassifications.

In conclusion, while LDA is a valuable tool for visualizing high-dimensional data, our experiment suggests that SVM might be a better choice for face recognition tasks given its superior performance in our tests. Further testing could help to confirm these findings and improve these models performance.